# Projeto - Análise de Risco de Crédito

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_path = r"C:\Users\Usuario\OneDrive\Documentos\Projetos Visual Studio Code\DSNP 3.0\Datasets\acquisition_train.csv"

cred_risk_df = pd.read_csv(data_path)

cred_risk_df.head()

,ids,target_default,score_1,score_2,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,...,external_data_provider_fraud_score,lat_lon,marketing_channel,profile_phone_number,reported_income,shipping_state,shipping_zip_code,profile_tags,user_agent,target_fraud
0,343b7e7b-2cf8-e508-b8fd-0a0285af30aa,False,1Rk8w4Ucd5yR3KcqZzLdow==,IOVu8au3ISbo6+zmfnYwMg==,350.0,101.800832,0.259555,108.427273,0.40,25033.92,...,645,"(-29.151545708122246, -51.1386461804385)",Invite-email,514-9840782,57849.0,BR-MT,17528,"{'tags': ['n19', 'n8']}",Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,NaN
1,bc2c7502-bbad-0f8c-39c3-94e881967124,False,DGCQep2AE5QRkNCshIAlFQ==,SaamrHMo23l/3TwXOWgVzw==,370.0,97.062615,0.942655,92.002546,0.24,NaN,...,243,"(-19.687710705798963, -47.94151536525154)",Radio-commercial,251-3659293,4902.0,BR-RS,40933,"{'tags': ['n6', 'n7', 'nim']}",Mozilla/5.0 (Linux; Android 5.0.2; SAMSUNG SM-...,NaN
2,669630dd-2e6a-0396-84bf-455e5009c922,True,DGCQep2AE5QRkNCshIAlFQ==,Fv28Bz0YRTVAT5kl1bAV6g==,360.0,100.027073,0.351918,112.892453,0.29,7207.92,...,65,"(-28.748023890412284, -51.867279334353995)",Waiting-list,230-6097993,163679.0,BR-RR,50985,"{'tags': ['n0', 'n17', 'nim', 'da']}",Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,NaN
3,d235609e-b6cb-0ccc-a329-d4f12e7ebdc1,False,1Rk8w4Ucd5yR3KcqZzLdow==,dCm9hFKfdRm7ej3jW+gyxw==,510.0,101.599485,0.987673,94.902491,0.32,NaN,...,815,"(-17.520650158450454, -39.75801139933186)",Waiting-list,261-3543751,1086.0,BR-RN,37825,{'tags': ['n4']},Mozilla/5.0 (Linux; Android 6.0; HTC One X10 B...,NaN
4,9e0eb880-e8f4-3faa-67d8-f5cdd2b3932b,False,8k8UDR4Yx0qasAjkGrUZLw==,+CxEO4w7jv3QPI/BQbyqAA==,500.0,98.474289,0.532539,118.126207,0.18,NaN,...,320,"(-16.574259446978008, -39.90990074785962)",Invite-email,102-3660162,198618.0,BR-MT,52827,"{'tags': ['pro+aty', 'n19', 'da', 'b19']}",Mozilla/5.0 (Linux; Android 7.0; Pixel C Build...,NaN


In [4]:
print('Dimensões do conjunto\n',
      f'\nEntradas: {cred_risk_df.shape[0]}',
      f'\nVariáveis: {cred_risk_df.shape[1]}')

print('\nNomes das colunas:')
cred_risk_df.columns

Dimensões do conjunto
 
Entradas: 45000 
Variáveis: 43

Nomes das colunas:


Index(['ids', 'target_default', 'score_1', 'score_2', 'score_3', 'score_4',
       'score_5', 'score_6', 'risk_rate', 'last_amount_borrowed',
       'last_borrowed_in_months', 'credit_limit', 'reason', 'income',
       'facebook_profile', 'state', 'zip', 'channel', 'job_name', 'real_state',
       'ok_since', 'n_bankruptcies', 'n_defaulted_loans', 'n_accounts',
       'n_issues', 'application_time_applied', 'application_time_in_funnel',
       'email', 'external_data_provider_credit_checks_last_2_year',
       'external_data_provider_credit_checks_last_month',
       'external_data_provider_credit_checks_last_year',
       'external_data_provider_email_seen_before',
       'external_data_provider_first_name',
       'external_data_provider_fraud_score', 'lat_lon', 'marketing_channel',
       'profile_phone_number', 'reported_income', 'shipping_state',
       'shipping_zip_code', 'profile_tags', 'user_agent', 'target_fraud'],
      dtype='object')

### Dicionário de Variáveis

Observando os nomes de colunas e as primeiras entradas, podemos inferir algumas coisas. Além disso, se analisarmos cada variável em seus valores únicos, podemos deduzir o signficado, além de já identificar problemas nos dados:

- `ids`: identificadores dos clientes, aparentemente são _strings_ aleatórias que não serão úteis para a análise;
- `score_1, score_2`: variáveis de pontuação codificadas;
- `score_3, score_4, score_5, score_6`: variáveis numéricas que dizem respeito a pontuação do cliente;
- `risk_rate`: taxa de risco (de inadimplência?) do cliente (entre 0 e 1);
- `last_amount_borrowed`: registro do valor do último empréstimo do cliente;
- `last_borrowed_in_months`: número de meses desde o último empréstimo (36., nan, 60.);
- `credit_limit`: limite de crédito para determinado cliente;
- `income`: renda anual do cliente;
- `state, zip, channel, job_name, real_state`: dados demográficos, trabalho dos clientes (codificadas);
- `n_bankruptcies`: número de falências registradas do cliente ( 0.,  1., nan,  2.,  3.,  4.,  5.);
- `n_defaulted_loans`: número de calotes nos empréstimos (0., 1., nan, 2., 3., 5.);
- `n_accounts`: número de contas (valor inteiro);
- `n_issues`: número de problemas (valor inteiro);
- `application_time_applied`: tempo de aplicação (dados em horas, não está claro a que se refere);
- `email`: provedor de e-mail (outlook, gmail, etc);
- `external_data_provider_credit_checks_last_2_year`: coluna com 0 ou nan. Checagem de crédito nos últimos dois anos;
- `external_data_provider_credit_checks_last_year, `external_data_provider_credit_checks_last_month`: Checagem de crédito no último ano (0., nan, 1.) e último mês (2, 1, 3, 0);
- `external_data_provider_email_seen_before`: E-mail visto antes (possui valores nan e discrepantes -999)
- `external_data_provider_first_name`: Primeiro nome do cliente;
- `external_data_provider_fraud_score`: Pontuação de fraude (não faz parte do escopo da nossa análise);
- `lat_lon`: tupla de valores de latitude e longitude;
- `marketing_channel`: canal de marketing (convite e-mail, radio, lista de espera, website, etc);
- `profile_phone_number`: número de telefone do cliente;
- `reported_income`: renda informada pelo cliente (float, mas poderia ser int);
- `shipping_state`, `shipping_zip_code`: estado de residência do cliente (BR-MG, BR-SP, etc) e CEP (int);
- `profile_tags`: lista com dicionários contendo tags de identificação (?) dos clientes ("{'tags': ['n19', 'n8']}", "{'tags': ['n6', 'n7', 'nim']}"). PS: dicionários formatados como string.
- `user_agent`: informações sobre pontos de acesso;
- `target_fraud`: variável que classifica uma transação fraudulenta ou não (fora do escopo do projeto);
- `target_default`: variável que classifica um cliente inadimplente ou não (alvo) (False, True, nan).

A análise dos valores únicos por coluna ajudou a identificar as colunas que já apresentavam problemas de valores discrepantes e ausentes, além daquelas cujos valores estão codificados. Como é uma análise feita manualmente, as células foram apagadas para não bagunçar o projeto.




### Valores únicos da variável-alvo

In [5]:
print('Valores na variável-alvo:')
cred_risk_df['target_default'].unique().tolist()

Valores na variável-alvo:


[False, True, nan]

In [9]:
cred_risk_df['target_default'].value_counts()

False    35080
True      6661
Name: target_default, dtype: int64

In [11]:
print('Quantidade de valores nulos no alvo: {}'.format(cred_risk_df['target_default'].isna().sum()))

Quantidade de valores nulos no alvo: 3259


Como podemos ver, nossa variável-alvo apresenta valores de verdadeiro, falso e valores ausentes que precisam ser tratados. Trata-se então de uma tarefa de classificação binária, em que a classe verdadeira representa os cliente inadimplentes e a classe falta os clientes em boa posição.

### Informações do conjunto de dados

In [6]:
print('Resumo de informações do conjunto, tipos e valores ausentes:\n')
cred_risk_df.info(verbose = True)

Resumo de informações do conjunto, tipos e valores ausentes:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 43 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   ids                                               45000 non-null  object 
 1   target_default                                    41741 non-null  object 
 2   score_1                                           44438 non-null  object 
 3   score_2                                           44438 non-null  object 
 4   score_3                                           44438 non-null  float64
 5   score_4                                           45000 non-null  float64
 6   score_5                                           45000 non-null  float64
 7   score_6                                           45000 non-null  float64
 8   risk_rate                         

### Tratamento de valores ausentes

In [7]:
print('Valores ausentes por coluna, ordenados de forma decrescente (%):')
round((cred_risk_df.isna().sum()) * 100 / cred_risk_df.__len__(), 2).sort_values(ascending = False)

Valores ausentes por coluna, ordenados de forma decrescente (%):


target_fraud                                        96.62
last_amount_borrowed                                66.57
last_borrowed_in_months                             66.57
ok_since                                            58.99
external_data_provider_credit_checks_last_2_year    50.28
external_data_provider_credit_checks_last_year      33.61
credit_limit                                        30.67
n_issues                                            25.65
facebook_profile                                     9.91
marketing_channel                                    7.95
job_name                                             7.41
target_default                                       7.24
external_data_provider_email_seen_before             4.96
lat_lon                                              3.03
user_agent                                           1.60
n_bankruptcies                                       1.55
n_defaulted_loans                                    1.28
reason        

Variáveis sem nenhum valor ausente:

- `profile_tags`
- `shipping_zip_code`
- `shipping_state`
- `reported_income`
- `profile_phone_number`
- `external_data_provider_credit_checks_last_month`
- `external_data_provider_fraud_score`
- `external_data_provider_first_name`
- `score_4`
- `score_5`
- `score_6`
- `email`
- `application_time_in_funnel`
- `application_time_applied`
- `ids`                  

Dentre as variáveis que possuem valores ausentes, algumas não possuem nenhum significado para nossa análise e serão removidas.  Particularmente, as variáveis que dizem respeito à tarefa de detecção de fraudes, que não faz parte do escopo desse projeto, serão removidas. São elas: `target_fraud` e `external_data_provider_fraud_score`. Outras variáveis simplesmente não possuem nenhum poder preditivo para análise (e.g nome ou e-mail que a pessoa usa, perfil do Facebook) ou são de difícil interpretabilidade (e.g pontos de acesso, horários, etc), como `ids`, `email`, `external_data_provider_first_name`, `profile_phone_number`, `facebook_profile`, `user_agent`, `application_time_applied`, `external_data_provider_credit_checks_last_year`, `external_data_provider_credit_checks_last_2_year`, `reason`, etc.

Dentre as variáveis codificadas, temos: `state, zip, channel, job_name, real_state, score_1 e score_2`. Para verificar se compensa ou não removê-las, podemos investigar as correlações que essas variáveis tem com o alvo. Se forem correlações significativas, podemos mantê-las no conjunto de dados. Para analisar as correlações, essas variáveis precisam ser numéricas. Assim irei utilizar da técnica de One-Hot Enconding naquelas que possuem poucos valores únicos, para transformar as categorias em novas colunas. Primeiro vamos verificar os valores únicos dessas colunas:

In [8]:
cred_risk_df[['state', 'zip', 'channel', 'job_name', 'real_state', 'score_1', 'score_2']].nunique()

state            50
zip             823
channel           1
job_name      32265
real_state        5
score_1           7
score_2          35
dtype: int64

Apenas nas variáveis `score_1` e `real_state`. A variável `channel` possui apenas um valor que por si só não parece indicar muito poder preditivo. Iremos removê-la também. Poderíamos argumentar que o fato do cliente ser inadimplente pode ter a ver com seu emprego. Vamos analisar se existe algum tipo de emprego, na coluna `job_name`, que indique essa relação. 

In [9]:
cred_risk_df.loc[cred_risk_df['target_default'] == True]['job_name'].value_counts()

mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtUhXGgZjtNQMnUXIWhIhmLVIVxoGY7TUDJ1FyFoSIaUIfpgwbYh438CvSsT5QB8=    19
mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtUhXGgZjtNQMnUXIWhIhmLVIVxoGY7TUDJ1FyFoSIbOy82w5K5LALfp4MHskDUE=    17
mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtUhXGgZjtNQMnUXIWhIhmLVIVxoGY7TUDJ1FyFoSIVf4E/iI7qK+Lfl5hxoWW2A=    16
mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtUhXGgZjtNQMnUXIWhIhmLVIVxoGY7TUDJ1FyFoSISzmgfFzT+qYrtqTHkJFqpA=    12
mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtUhXGgZjtNQMnUXIWhIhmLVIVxoGY7TUDJ1FyFoSIUGp9a+9oBSLvyI5Jdz9fNg=    12
                                                                                                                ..
mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtUhXGgZjtNQMnUXIWhIhtW0k8u7Ytcx6b4rwm8tpJoW9+H8TEki+3Dz1vZ4h/cY=     1
mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtUhXGgZjtNQMnUXIWhIhmLVIVxoGY7TUDJ1FyFoSIVwwRWY97FN+sS2HV7/nleQ=     1
mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtUhXGgZjtNQMnUXIWhIhmLVIVxoGY7TUDJ1F

A coluna `job_name` possui 32265 títulos de emprego diferentes. Das entradas classificadas cuja variável-alvo tem valor True, vemos que o emprego que mais se repete o faz apenas 19 vezes. Não parece haver nenhum poder preditivo nessa variável, i.e, nenhum emprego específico que indique uma maior probabilidade de inadimplência. O emprego que aparece 19 vezes pode ser apenas mais comum na vida real do que o que aparece uma única vez (e.g comerciante vs piloto de avião). Ainda assim, a proporção é muito pequena e a variável será removida.

Vamos analisar mais algumas variáveis, visando conhecer suas distribuições estatísticas e valores únicos. Assim, poderemos escolher quais iremos manter, quais remover e quais imputar valores.

In [14]:
print('Variável ok_since:\n')

print(cred_risk_df['ok_since'].describe().round(2), '\n'), print(cred_risk_df['ok_since'].unique())

Variável ok_since:

count    18455.00
mean        35.07
std         21.66
min          0.00
25%         17.00
50%         32.00
75%         50.00
max        141.00
Name: ok_since, dtype: float64 

[ nan  50.  41.  11.  35.  16.  19.  23.  43.  17.  29.  18.   8.  56.
  38.  93.  27.  44.  13.  66.  33.  68.  12.  30.  24.  21.   7.  63.
  51.  31.  73.  49.   4.  75.  48.  14.  76.  36.   5.  46.  34.  59.
  61.  55.  84.  28.   6.  67.  39.  52.  72.  10.  45.  32.  62.   0.
  58.  15.  78.  70.   9.  22.  80.  20.  37.   3.  53.  60.  74.  26.
  47.   1.  42.  57.  40.  54.  65.  81.  69.  25.  77.  71.  64.  86.
  79.  82.   2.  83.  85. 103. 135.  97. 101.  90. 130. 141. 108.  88.
  95. 122.  87.]


(None, None)

In [15]:
cred_risk_df['last_amount_borrowed'].describe().round(2)

count    15044.00
mean     13782.34
std       8077.57
min       1005.18
25%       7519.50
50%      12023.46
75%      19220.27
max      35059.60
Name: last_amount_borrowed, dtype: float64

In [44]:
cred_risk_df['external_data_provider_email_seen_before'].describe()

count    42767.000000
mean        12.534781
std        126.530487
min       -999.000000
25%         11.000000
50%         27.000000
75%         43.000000
max         59.000000
Name: external_data_provider_email_seen_before, dtype: float64

Variáveis para imputar valores:

* ok_since: mediana
* last_amount_borrowed: mediana
* credit_limit: mediana
* n_issues: mediana
* marketing_channel: moda

In [25]:
cred_risk_df['application_time_in_funnel']

0        444
1        346
2          6
3        406
4        240
        ... 
44995     72
44996     51
44997    476
44998     35
44999    414
Name: application_time_in_funnel, Length: 45000, dtype: int64

In [27]:
df_clean = cred_risk_df.copy()

cols_to_drop = ['target_fraud', 'external_data_provider_fraud_score',
                'ids', 'email', 'external_data_provider_first_name',
                'profile_phone_number', 'profile_tags', 'facebook_profile',
                'user_agent', 'state', 'zip', 'job_name', 'channel',
                'application_time_applied', 'last_borrowed_in_months',
                'external_data_provider_credit_checks_last_year',
                'external_data_provider_credit_checks_last_2_year']

for col in cols_to_drop:
    if col in df_clean.columns.tolist():
        
        df_clean.drop(col, axis = 1, inplace = True)

print('Nova Dimensão do Conjunto')
print(f'\nEntradas: {df_clean.shape[0]}',
      f'\nVariáveis: {df_clean.shape[1]}')

Nova Dimensão do Conjunto

Entradas: 45000 
Variáveis: 26
